# Load data

In [1]:
import pandas as pd

# Define the path to the dataset
file_path = "../data/MachineLearningRating_v3.txt"

# Load the data
df = pd.read_csv(file_path, delimiter='|')

# Show the shape and first few rows
print("Shape of dataset:", df.shape)
df.head()

/tmp/ipykernel_94363/401646834.py:7: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, delimiter='|')


Shape of dataset: (1000098, 52)


,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


# checking missed values

In [2]:
# Check for missing values
missing_summary = df.isnull().sum().sort_values(ascending=False)
print("Missing values:\n", missing_summary[missing_summary > 0])

# Data types
df.dtypes


Missing values:
 NumberOfVehiclesInFleet    1000098
CrossBorder                 999400
CustomValueEstimate         779642
Rebuilt                     641901
Converted                   641901
WrittenOff                  641901
NewVehicle                  153295
Bank                        145961
AccountType                  40232
Gender                        9536
MaritalStatus                 8259
VehicleType                    552
make                           552
mmcode                         552
Model                          552
Cylinders                      552
bodytype                       552
kilowatts                      552
NumberOfDoors                  552
VehicleIntroDate               552
cubiccapacity                  552
CapitalOutstanding               2
dtype: int64


UnderwrittenCoverID           int64
PolicyID                      int64
TransactionMonth             object
IsVATRegistered                bool
Citizenship                  object
LegalType                    object
Title                        object
Language                     object
Bank                         object
AccountType                  object
MaritalStatus                object
Gender                       object
Country                      object
Province                     object
PostalCode                    int64
MainCrestaZone               object
SubCrestaZone                object
ItemType                     object
mmcode                      float64
VehicleType                  object
RegistrationYear              int64
make                         object
Model                        object
Cylinders                   float64
cubiccapacity               float64
kilowatts                   float64
bodytype                     object
NumberOfDoors               

# Define risk Metrics

In [3]:
# Create new columns
df['ClaimOccurred'] = df['TotalClaims'] > 0
df['Margin'] = df['TotalPremium'] - df['TotalClaims']

# Sanity check
df[['TotalPremium', 'TotalClaims', 'ClaimOccurred', 'Margin']].head()


,TotalPremium,TotalClaims,ClaimOccurred,Margin
0,21.929825,0.0,False,21.929825
1,21.929825,0.0,False,21.929825
2,0.000000,0.0,False,0.000000
3,512.848070,0.0,False,512.848070
4,0.000000,0.0,False,0.000000


# Hypothesis

In [4]:
# Group by province
province_group = df.groupby('Province')

# Compute metrics
risk_by_province = province_group.agg(
    ClaimFrequency=('ClaimOccurred', 'mean'),
    ClaimSeverity=('TotalClaims', lambda x: x[x > 0].mean()),
    AverageMargin=('Margin', 'mean'),
    Count=('TotalClaims', 'count')
).sort_values('ClaimFrequency', ascending=False)

risk_by_province


,ClaimFrequency,ClaimSeverity,AverageMargin,Count
Province,,,,
Gauteng,0.003356,22243.878396,-13.558894,393865
KwaZulu-Natal,0.002845,29609.487473,-6.433598,169781
Limpopo,0.002698,15171.294187,20.971484,24836
North West,0.002436,16963.467035,10.958832,143287
Mpumalanga,0.002428,15979.553421,15.016059,52718
Western Cape,0.002166,28095.849881,-3.414689,170796
Eastern Cape,0.001648,27128.533277,25.833240,30336
Free State,0.001358,32265.661085,20.550805,8099
Northern Cape,0.001254,11186.313596,35.590527,6380


# Anova for claim Frequency

In [5]:
from scipy.stats import f_oneway

# Get ClaimOccurred as a list of series for each province
province_freqs = [group['ClaimOccurred'].values for name, group in df.groupby('Province') if len(group) > 10]

# ANOVA test
f_stat_freq, p_val_freq = f_oneway(*province_freqs)
print("ANOVA for Claim Frequency")
print("F-statistic:", f_stat_freq)
print("p-value:", p_val_freq)


ANOVA for Claim Frequency
F-statistic: 13.025099894182974
p-value: 5.91210036318191e-19


# Anova for claim severity

In [6]:
# Filter to only rows with claims
df_with_claims = df[df['ClaimOccurred'] == True]

# Prepare severity values per province
province_severities = [group['TotalClaims'].values for name, group in df_with_claims.groupby('Province') if len(group) > 10]

# ANOVA test
f_stat_sev, p_val_sev = f_oneway(*province_severities)
print("\nANOVA for Claim Severity")
print("F-statistic:", f_stat_sev)
print("p-value:", p_val_sev)



ANOVA for Claim Severity
F-statistic: 5.3945453647231085
p-value: 3.7043515771514927e-06


# Anova for Margin

In [7]:
# Prepare margin values per province
province_margins = [group['Margin'].values for name, group in df.groupby('Province') if len(group) > 10]

# ANOVA test
f_stat_margin, p_val_margin = f_oneway(*province_margins)
print("\nANOVA for Margin")
print("F-statistic:", f_stat_margin)
print("p-value:", p_val_margin)



ANOVA for Margin
F-statistic: 3.2226132627264557
p-value: 0.0011450081247588997


# for postal codes

In [8]:
# Claim Frequency by PostalCode
zipcode_freqs = [group['ClaimOccurred'].values for _, group in df.groupby('PostalCode') if len(group) > 10]
f_freq_zip, p_freq_zip = f_oneway(*zipcode_freqs)

# Claim Severity by PostalCode (claims only)
zipcode_claims = df[df['ClaimOccurred'] == True]
zipcode_severity = [group['TotalClaims'].values for _, group in zipcode_claims.groupby('PostalCode') if len(group) > 10]
f_sev_zip, p_sev_zip = f_oneway(*zipcode_severity)

# Margin by PostalCode
zipcode_margin = [group['Margin'].values for _, group in df.groupby('PostalCode') if len(group) > 10]
f_margin_zip, p_margin_zip = f_oneway(*zipcode_margin)

# Print results
print("Postal Code - Claim Frequency: F =", f_freq_zip, "| p =", p_freq_zip)
print("Postal Code - Claim Severity: F =", f_sev_zip, "| p =", p_sev_zip)
print("Postal Code - Margin: F =", f_margin_zip, "| p =", p_margin_zip)


Postal Code - Claim Frequency: F = 1.71562707640108 | p = 1.2263452248289006e-34
Postal Code - Claim Severity: F = 2.5618324298081614 | p = 3.926584200071599e-08
Postal Code - Margin: F = 0.9104309561502105 | p = 0.970313591024769


# A/B Testing on Gender

In [1]:
import pandas as pd

# Load your data again (if needed)
df = pd.read_csv("../data/MachineLearningRating_v3.txt", delimiter="|")

/tmp/ipykernel_102188/3073931425.py:4: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/MachineLearningRating_v3.txt", delimiter="|")


In [2]:
# Create the computed columns
df['ClaimOccurred'] = df['TotalClaims'] > 0
df['Margin'] = df['TotalPremium'] - df['TotalClaims']
df[['TotalClaims', 'ClaimOccurred', 'Margin']].head()

,TotalClaims,ClaimOccurred,Margin
0,0.0,False,21.929825
1,0.0,False,21.929825
2,0.0,False,0.000000
3,0.0,False,512.848070
4,0.0,False,0.000000


In [3]:
from scipy.stats import ttest_ind

# Remove missing/undefined gender rows
gender_df = df[df['Gender'].notnull()]

# Separate male and female
male = gender_df[gender_df['Gender'].str.lower().str.startswith('m')]
female = gender_df[gender_df['Gender'].str.lower().str.startswith('f')]

# Claim Frequency
freq_male = male['ClaimOccurred']
freq_female = female['ClaimOccurred']
t_freq_gender, p_freq_gender = ttest_ind(freq_male, freq_female)

# Claim Severity
sev_male = male[male['ClaimOccurred']]['TotalClaims']
sev_female = female[female['ClaimOccurred']]['TotalClaims']
t_sev_gender, p_sev_gender = ttest_ind(sev_male, sev_female)

# Margin
margin_male = male['Margin']
margin_female = female['Margin']
t_margin_gender, p_margin_gender = ttest_ind(margin_male, margin_female)

# Results
print("Gender - Claim Frequency: t =", t_freq_gender, "| p =", p_freq_gender)
print("Gender - Claim Severity: t =", t_sev_gender, "| p =", p_sev_gender)
print("Gender - Margin: t =", t_margin_gender, "| p =", p_margin_gender)


Gender - Claim Frequency: t = -inf | p = nan
Gender - Claim Severity: t = -0.4190662866061044 | p = 0.6760156776445874
Gender - Margin: t = -0.21038507904864107 | p = 0.833368001967537
